Extracting youtube video and its audio  
using pytube package of python.

In [ ]:
! pip install pytube
! pip install moviepy

In [3]:
! pip install --q --upgrade moviepy pytube git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

## Task 1.1 
getting audio from video

In [1]:
from pytube import YouTube
from moviepy.editor import VideoFileClip


getting stream value of video from youtube using pytube

In [2]:
url = 'https://www.youtube.com/watch?v=Sby1uJ_NFIY'
# url = 'https://www.youtube.com/watch?v=CeSAjK2CBEA&t=81s'

yt_vid = YouTube(url) 

# progressive streams have video and audigo in a single file 
# but are only for upto 720p
yt_vid.streams.filter(progressive=True)


[<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">]

download video from youtube url

In [3]:
def get_video(yt_url):
    try:
        video = YouTube(yt_url)
        video.streams.get_by_itag(22).download(filename='video.mp4')
        print('Video succesfully downloaded from Youtube')
    except Exception as e:
        print(f'Failed to download Youtube video \nerror : {e}')


Extracting audio from downloaded video

In [4]:
def audio_from_video(video_path):
    try:
        video = VideoFileClip(video_path)
        audio = video.audio
        audio.write_audiofile('audio.wav')   
        video.close()
        audio.close()
    except Exception as e:
        print(f'Failed to extract audio from video \nerror : {e}')


In [5]:
url = 'https://www.youtube.com/watch?v=CeSAjK2CBEA&t=81s'
video_path = './video.mp4'

get_video(url)
audio_from_video(video_path)


Video succesfully downloaded from Youtube
MoviePy - Writing audio in audio.wav


MoviePy - Done.


## Task 1.2 
Transcription of audio

audio processing

In [6]:
import wave

In [7]:
obj = wave.open('./audio.wav')

In [8]:
print("number of channels : ", obj.getnchannels())
print("sample width : ", obj.getsampwidth())
print("frame rate : ", obj.getframerate())
print("Number of frames : ", obj.getnframes())
print("parameters", obj.getparams())

obj.close()

number of channels :  2
sample width :  2
frame rate :  44100
Number of frames :  7769979
parameters _wave_params(nchannels=2, sampwidth=2, framerate=44100, nframes=7769979, comptype='NONE', compname='not compressed')


In [9]:
t_audio = obj.getnframes() / obj.getframerate()
t_audio # seconds

176.19

In [ ]:
frames = obj.readframes(-1)
print(type(frames), type(frames[0]))
print(len(frames))

plotting the wave

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np

In [ ]:

samp_freq =  obj.getframerate()
n_samples = obj.getnframes()
signal_wave = obj.readframes(-1)

obj.close()

t_audio = n_samples / samp_freq
type(signal_wave)
signal_wave


b''

Task 1.2

In [11]:
# importing dependencies
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [12]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

device

'cpu'

using openai whisper model (tiny)

In [13]:
model_id = "openai/whisper-tiny"

In [14]:
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
pipeline = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

pipeline

In [18]:
audio = 'audio.wav'

In [19]:
text_audio = pipeline(audio)

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


getting transcribed text

In [20]:
text_audio['text']

" When you grow up you tend to get told that the world is the way it is and your life is just to live your life inside the world, try not to bash into the walls too much. But life can be much broader once you discover one simple fact, and that is everything around you that you call life was made up by people that were no smarter than you. And you can change it, you can influence it, you can build your own things that other people can use. Good evening. I'm Steve Jobs from Apple Computer, We're very glad to be here tonight. I think you always had to be a little different to buy an Apple computer. And I think you still have to think differently to buy an Apple computer. And I think the people that do buy this. still have to think differently to buy and have a computer. And I think the people that do buy them, they are the people that are not just out to get a job done, they're out to change the world. What a computer is to me is it's the most remarkable tool that we've ever come up with.

In [22]:
chunks = text_audio['chunks']

In [110]:
chunks_count = len(chunks)

In [111]:
chunk_id = []
timestamps = []
texts = []
start_time = []
end_time = []


for i in range(0, chunks_count):
  chunk_id.append(i)
  texts.append(chunks[i]['text'])
  start_time.append(chunks[i]['timestamp'][0])
  end_time.append(chunks[i]['timestamp'][1])



In [112]:
chunk_length = []
for i in range(0, chunks_count-1):
    chunk_length.append(round(end_time[i] - start_time[i], 3))

In [113]:
output = list(zip(chunk_id, chunk_length, texts, start_time, end_time))

In [114]:
sample_output_list = []
for sublist in output:
    chunk_dict = {
        "chunk_id": sublist[0],
        "chunk_length": sublist[1],
        "text": sublist[2],
        "start_time": sublist[3],
        "end_time": sublist[4]
    }
    sample_output_list.append(chunk_dict)

In [127]:
file = open('trascribe.txt', "w")
for output in sample_output_list:
    file.write(str(output)+"\n\n")
file.close()

In [129]:
for outputs in sample_output_list:
    print(str(outputs) + "\n")

{'chunk_id': 0, 'chunk_length': 14.82, 'text': ' When you grow up you tend to get told that the world is the way it is and your life is just to live your life inside the world,', 'start_time': 0.0, 'end_time': 14.82}

{'chunk_id': 1, 'chunk_length': 2.7, 'text': ' try not to bash into the walls too much.', 'start_time': 14.82, 'end_time': 17.52}

{'chunk_id': 2, 'chunk_length': 7.12, 'text': ' But life can be much broader once you discover one simple fact, and that is everything around you that you', 'start_time': 17.52, 'end_time': 24.64}

{'chunk_id': 3, 'chunk_length': 7.1, 'text': ' call life was made up by people that were no smarter than you. And you can change it, you', 'start_time': 24.64, 'end_time': 31.74}

{'chunk_id': 4, 'chunk_length': 4.26, 'text': ' can influence it, you can build your own things that other people can use.', 'start_time': 31.74, 'end_time': 36.0}

{'chunk_id': 5, 'chunk_length': 5.84, 'text': " Good evening. I'm Steve Jobs from Apple Computer, We're very